In [1]:
class var_ewma:
    def __init__(self,data,weights):
        self.data=data
        self.weights=weights 
    
    def port_returns(self):
        import pandas as pd
        import numpy as np
        returns=self.data.pct_change().dropna()
        total=[]
        for b in range(len(returns)):
            array_returns=np.transpose(np.array(returns.iloc[b,:]))
            array_weights=np.array(self.weights)
            total.append(np.dot(array_returns,array_weights))
        self.total_return=total
        self.returns=returns
        return np.array(total)
    
    def ewma(self,conf_level,lambdaa): #Ewma VaR
        import pandas as pd
        import numpy as np
        port_return=pd.DataFrame(self.total_return)
        port_return.index=self.returns.index
        ret_2=port_return**2
        ret_2_sorted=ret_2.sort_index(ascending=False)
        peso=[]
        inicial=1-lambdaa
        for j in range(len(ret_2_sorted)):
            peso.append(inicial)
            inicial=peso[j]*lambdaa
        peso_lambda=pd.Series(peso)
        peso_lambda.index=ret_2_sorted.index
        multipl=peso_lambda*pd.Series(ret_2_sorted.iloc[:,0])
        sigma_ewma=np.sqrt(np.sum(multipl))
        import scipy.stats as stats
        z=stats.norm.ppf(1-conf_level,0,1)
        var_ewma=(z*sigma_ewma)+np.mean(self.total_return)
        print("VaR Ewma is: "+str(var_ewma)+".")

In [4]:
import pandas as pd
import numpy as np
dataaa=pd.read_excel("portafolioo.xlsx",sheet_name="Hoja5",index_col=0)
pesooos=pd.read_excel("portafolioo.xlsx",sheet_name="Hoja6")
dataaa.head()

,COCA COLA,EXXON MOBIL,JOHNSON & JOHNSON,JP MORGAN,APPLE
FECHA,,,,,
2018-01-02,39.689281,66.545746,124.394119,95.053246,41.075405
2018-01-03,39.602119,67.852707,125.582382,95.150124,41.068253
2018-01-04,40.159908,67.946625,125.573448,96.513199,41.259010
2018-01-05,40.151192,67.891838,126.609863,95.893608,41.728756
2018-01-08,40.090179,68.197075,126.770683,96.035217,41.573765


In [5]:
pesooos.head()

,Pesos
0,0.15
1,0.21
2,0.18
3,0.14
4,0.32


In [6]:
objeto_var=var_ewma(dataaa,pesooos)
retornos_port=objeto_var.port_returns()                                    
ewma=objeto_var.ewma(0.99, 0.90)

VaR Ewma is: -0.04114585240725244.
